# 推薦システム
- このファイルは、ユーザーに合わせて好みの単語帳を予測し、各ユーザーへ推薦を行うためのモジュールです。
- 推薦アルゴリズムは「協調フィルタリング」を利用し、Surpriseライブラリを利用します。
- 余裕があれば、Google Analytics、Google Optimizeを利用し、機械学習モデルのABテストを行います。

## 手順
1. 各種ライブラリのimport
2. DBの接続
3. userテーブル取得
4. wordbookテーブル取得
5. 評価値行列の作成
6. 次元削減 (NMF)
7. 推薦アルゴリズムの作成 (ユーザーベース協調フィルタリング)
8. scikit-surpriseを用いた協調フィルタリング
9. 6、7、8のモデル評価

※ 一般的に3~6を「前処理」、7~8は「機械学習」と呼びます。

### 1. 各種ライブラリのimport

In [1]:
import os
import sqlite3
import psycopg2
import pickle
# recommender
import numpy as np
from scipy.spatial.distance import cosine
import pandas as pd
from sklearn.decomposition import NMF
# scikit-surprise
from surprise import Dataset, Reader, SVD, NMF
from surprise.model_selection import GridSearchCV

### 2. DBの接続

In [2]:
# working directory path
working_dir = os.path.abspath('__file__')

# project directory path
project_dir = os.path.dirname \
    (os.path.dirname(os.path.dirname(working_dir)))

# db path
db_path = os.path.join(project_dir, 'db.sqlite3')

# connect db
if 'db.sqlite3' in os.listdir(project_dir):
    con = sqlite3.connect(db_path)  # sqlite3
else:
    con = psycopg2.connect(dbname='wordbook')  # PostgreSQL

### 3. ユーザidの取得

In [3]:
# user
q_user = '''
SELECT
    id AS uid
FROM
    user
'''

# user DataFrame
user_id = pd.read_sql(q_user, con)

user_id

,uid
0,1
1,16
2,15
3,17
4,18
5,26
6,25


### 4. 商品idの取得

In [4]:
# wordbook
q_wordbook = '''
SELECT
    id AS iid
FROM
    note AS n
'''

# wordbook DataFrame
item_id = pd.read_sql(q_wordbook, con)

item_id

,iid
0,1
1,13
2,14
3,15
4,16
5,17
6,19
7,31
8,33
9,36


### 5. 評価値行列の作成

In [8]:
%%time

# sql
likes_query = '''
SELECT
    s.user_id AS uid
    , s.note_id AS iid
FROM
    star AS s
;

'''

# 空の評価値行列
col = ['uid', 'iid', 'r_ui']
R = pd.DataFrame([], columns=col)

# いいね判定で利用する行列
likes = pd.read_sql(likes_query, con)

# 評価値行列の作成
for uid in user_id.uid:
    for iid in item_id.iid:
        # user_id, item_idで値を取得
        r_ui = likes[(likes['uid'] == uid) & (likes['iid'] == iid)]
        # いいねしている場合はTrue
        if r_ui.empty:
            rating = pd.Series([uid, iid, 1], index=col)
            R = R.append(rating, ignore_index=True)
        else:
            rating = pd.Series([uid, iid, 2], index=col)
            R = R.append(rating, ignore_index=True)

R.sample(10)

CPU times: user 299 ms, sys: 5.55 ms, total: 304 ms
Wall time: 302 ms


,uid,iid,r_ui
45,15,31,2
129,25,28,1
130,25,32,1
84,18,33,1
9,1,36,1
67,17,37,1
31,16,12,1
80,18,16,1
43,15,17,2
78,18,14,1


### scikit-surpriseのDatasetを作成

In [13]:
# Datasetで扱うReaderを定義
reader = Reader(rating_scale=(1,2))

# Dataset
dataset = Dataset.load_from_df(R, reader)

True

### アルゴリズムの設定とデータ準備

In [10]:
%%time

# data
trainset = dataset.build_full_trainset()
testset = trainset.build_anti_testset()

# model
alg = SVD()
alg.fit(trainset)

# prediction
prediction = alg.test(testset)

pd.DataFrame(prediction)

CPU times: user 16 ms, sys: 1.4 ms, total: 17.4 ms
Wall time: 16.3 ms


""


### 

In [5]:
# rating matrix
# U = '''
# SELECT
#     u.username
# FROM
#     user AS u
#     INNER JOIN (
#         SELECT
#             *
#         FROM
#             star AS s
#             INNER JOIN
#                 note AS n
#                 ON s.note_id == n.id
#         ) AS s
#         ON u.id == s.user_id
# ;

# '''

# V = '''
# SELECT
#     n.title
# FROM
#     note AS n
#     INNER JOIN
#         star AS s
#         ON n.id == s.note_id
# ;

# '''


# index and column of rating matrix
index = user['username']
columns = wordbook['title']

# rating matrix DataFrame
# R = pd.read_sql(rating_matrix, con, index=index, columns=columns)

pd.read_sql(U, con)
pd.read_sql(V, con)
# df

,title
0,Django
1,Django
2,Django
3,Django
4,test
5,hoge
6,vim基礎
7,vim基礎
8,SQLite
9,SQLite


In [8]:
"""sample data"""

# matrix dimention parameter
user_num = len(user)
item_num = len(wordbook)

# star probability
p = [.8, .2]

# create rating matrix function
def create_user_vector():
    user_vector = np.random.choice(2, item_num, p=p)
    return user_vector

# create matrix
R = []

for i in range(user_num):
    vector = create_user_vector()
    R.append(vector)

# DataFrame
R = pd.DataFrame(R, index=index, columns=columns)

R

title,Django,test,hoge,asdf,vim基礎,SQLite,VScode 拡張機能,JavaScirpt 基礎,VScode コマンド,AWS,test3,test2,Python,shell 基礎,マーケティング 入門（ちきりん）,推薦システム 入門,Mac ショートカット,Git 基礎 (Mac),マーケティング 入門
username,,,,,,,,,,,,,,,,,,,
admin,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,1
test,0,0,1,0,0,0,1,1,0,0,1,0,1,1,0,0,1,0,0
hoge,0,1,1,1,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0
test2,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0
test3,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0
てすと,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
てすてす,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [12]:
R.describe()

title,Django,test,hoge,asdf,vim基礎,SQLite,VScode 拡張機能,JavaScirpt 基礎,VScode コマンド,AWS,test3,test2,Python,shell 基礎,マーケティング 入門（ちきりん）,推薦システム 入門,Mac ショートカット,Git 基礎 (Mac),マーケティング 入門
count,7.0,7.000000,7.000000,7.000000,7.000000,7.0,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.0,7.000000,7.0,7.000000
mean,0.0,0.285714,0.285714,0.142857,0.142857,0.0,0.428571,0.428571,0.142857,0.142857,0.571429,0.285714,0.428571,0.285714,0.142857,0.0,0.714286,0.0,0.142857
std,0.0,0.487950,0.487950,0.377964,0.377964,0.0,0.534522,0.534522,0.377964,0.377964,0.534522,0.487950,0.534522,0.487950,0.377964,0.0,0.487950,0.0,0.377964
min,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
25%,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.500000,0.0,0.000000
50%,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.0,0.000000
75%,0.0,0.500000,0.500000,0.000000,0.000000,0.0,1.000000,1.000000,0.000000,0.000000,1.000000,0.500000,1.000000,0.500000,0.000000,0.0,1.000000,0.0,0.000000
max,0.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,0.0,1.000000


### 6. 次元削減 (NMF)

In [23]:
# NMF
k = item_num
nmf = NMF(n_components=k, init='random', random_state=42)

# columns of R^
col = [f'NMF{i}' for i in range(1, (k + 1))]

# R^ = W * H
W = nmf.fit_transform(R)
H = nmf.n_components
R_ = pd.DataFrame(np.dot(W, H), index=index, columns=columns)

R_

title,Django,test,hoge,asdf,vim基礎,SQLite,VScode 拡張機能,JavaScirpt 基礎,VScode コマンド,AWS,test3,test2,Python,shellでファイル実行,マーケティング 入門（ちきりん）,推薦システム 入門,Mac ショートカット,Git 基礎 (Mac)
username,,,,,,,,,,,,,,,,,,
admin,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000e+00
test,0.000000,0.000000,0.000000,0.000000e+00,1.435015e+01,1.864540e-15,1.888367e-15,0.000000e+00,0.000000,0.000000,0.000000,0.000000,22.098316,0.000000,5.808168e-14,0.000000e+00,0.0,0.000000e+00
hoge,0.000000,0.000155,9.470011,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.337332,1.311764,0.000000,0.000000,0.000000,0.000000,0.000000e+00,5.908874e+00,0.0,0.000000e+00
test2,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,7.277327,0.000000,0.000000,0.000000,0.000000e+00,2.783133e-09,0.0,0.000000e+00
test3,14.822259,0.000000,0.000000,1.721259e+01,0.000000e+00,3.730041e-09,0.000000e+00,1.148374e-09,0.000000,0.000000,0.000000,9.592122,0.000000,3.926022,0.000000e+00,0.000000e+00,0.0,4.362072e-10
てすと,35.801831,0.000000,0.000000,9.858090e-09,0.000000e+00,3.954765e-08,0.000000e+00,5.480970e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,1.755039e+01
てすてす,29.724245,38.918377,0.000000,2.672849e-02,3.715158e-09,2.212423e+01,1.464517e+01,0.000000e+00,0.000111,0.000000,0.000000,24.091676,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,6.730858e-03


In [14]:
a = R_.loc['admin', 'NMF12']
a

False

### 7. 推薦アルゴリズムの作成 (ユーザーベース協調フィルタリング)

- 「ユーザベース協調フィルタリング」を用いる

In [20]:
def calc_distance(u_index, u_matrix):
    """ユーザベース協調フィルタリング"""

    # user_iの評価値ベクトル
    pd_type = pd.core.frame.DataFrame
    if type(u_matrix) == pd_type:
        ratings = u_matrix.iloc[u_index, :]
    else:
        ratings = u_matrix[u_index]
    # 最後の推薦で扱う、i`temと同じ次元数の推薦ベクトルを定義
    similarity = np.zeros(len(ratings))

    # user_iと、各userの評価値ベクトルからコサイン類似度を求める
    for c_index in range(len(u_matrix)):
        # user_cの評価値べクトル
        if type(u_matrix) == pd_type:
            compare_ratings = u_matrix.iloc[c_index, :]
        else:
            compare_ratings = u_matrix[c_index]
        # c_indexがuser_iと同じindexの場合、計算しない
        if u_index == c_index:
            continue
        # コサイン類似度
        cosine_simirarity = 1. - cosine(ratings, compare_ratings)
        # コサイン類似度をuser_cの評価値ベクトルと掛け合わせる
        user_ratings = cosine_simirarity * compare_ratings
        # 掛け合わせたモノを推薦ベクトルに足す
        similarity += user_ratings
    return similarity

# model test
test_df = calc_distance(0, R)
pd.DataFrame(test_df).T

title,Django,test,hoge,asdf,vim基礎,SQLite,VScode 拡張機能,JavaScirpt 基礎,VScode コマンド,AWS,test3,test2,Python,shellでファイル実行,マーケティング 入門（ちきりん）,推薦システム 入門,Mac ショートカット,Git 基礎 (Mac)
test,0.566947,0.25,0.0,0.0,0.566947,0.566947,0.25,0.0,0.0,1.144297,0.566947,0.57735,0.82735,0.25,0.0,0.566947,0.566947,0.0


### 8. 6、7のモデル評価

In [16]:
# check model
R_ = []

r = calc_distance(0, R)

for i in range(len(R)):
    R_.append(calc_distance(i, R))

# columns
col = [f'NMF{i}' for i in range(1, len(item_num + 1))]

# DataFrame
pd.DataFrame(R_, index=index, columns=col)

title,Django,test,hoge,asdf,vim基礎,SQLite,VScode 拡張機能,JavaScirpt 基礎,VScode コマンド,AWS,test3,test2,Python,shellでファイル実行,マーケティング 入門（ちきりん）,推薦システム 入門,Mac ショートカット,Git 基礎 (Mac)
username,,,,,,,,,,,,,,,,,,
admin,0.000000,0.408248,0.353553,0.316228,0.0,0.353553,0.000000,0.316228,0.000000,0.000000,0.353553,0.000000,0.669781,0.000000,0.316228,0.000000,0.408248,0.724476
test,0.288675,0.288675,0.408248,0.258199,0.0,0.000000,0.000000,0.546874,0.000000,0.288675,0.000000,0.288675,0.258199,0.288675,0.258199,0.288675,0.288675,0.666447
hoge,0.288675,0.288675,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.288675,0.000000,0.288675,0.000000,0.000000
test2,0.000000,0.288675,0.000000,0.223607,0.0,0.000000,0.000000,0.223607,0.000000,0.000000,0.000000,0.000000,0.223607,0.000000,0.223607,0.000000,0.288675,0.512282
test3,0.000000,0.288675,0.000000,0.000000,0.0,0.000000,0.288675,0.000000,0.288675,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.288675,0.288675,0.288675
てすと,0.000000,0.258199,0.539835,0.000000,0.0,0.223607,0.000000,0.223607,0.000000,0.223607,0.223607,0.223607,0.223607,0.000000,0.000000,0.000000,0.481806,0.574427
てすてす,0.000000,0.000000,0.353553,0.223607,0.0,0.000000,0.000000,0.223607,0.000000,0.000000,0.000000,0.000000,0.223607,0.000000,0.223607,0.000000,0.000000,0.577160


### 9. 推薦モデルをpickleでオブジェクト化し、Djangoに組み込む